# Parallel Computing Lab 1 - Logic Gates Simulation
ECSE 420
- Eric Chao
- Chad Spector

**Note:** Notebook assumes input_X and sol_X .txt files are included in local directory.

## Dependencies: `compareResults.c`

### Source Code

In [30]:
%%writefile compareResults.cu
#include <stdio.h>
#include <stdlib.h>


void compareFiles(char *file_name1, char *file_name2) 
{ 
//get from https://www.tutorialspoint.com/c-program-to-compare-two-files-and-report-mismatches
FILE* fp1 = fopen(file_name1, "r");
FILE* fp2 = fopen(file_name2, "r");
    // fetching character of two file 
    // in two variable ch1 and ch2 
    char ch1 = getc(fp1); 
    char ch2 = getc(fp2); 
  
    // error keeps track of number of errors 
    // pos keeps track of position of errors 
    // line keeps track of error line 
    int error = 0, pos = 0, line = 1; 
  
    // iterate loop till end of file 
    while (ch1 != EOF && ch2 != EOF) 
    { 
        pos++; 
  
        // if both variable encounters new 
        // line then line variable is incremented 
        // and pos variable is set to 0 
        if (ch1 == '\n' && ch2 == '\n') 
        { 
            line++; 
            pos = 0; 
        } 
  
        // if fetched data is not equal then 
        // error is incremented 
        if (ch1 != ch2) 
        { 
            error++; 
            printf("Line Number : %d \tError"
               " Position : %d \n", line, pos); 
        } 
  
        // fetching character until end of file 
        ch1 = getc(fp1); 
        ch2 = getc(fp2); 
    } 
  
    printf("Total Errors : %d\t", error); 
} 

int main(int argc, char *argv[]){

    if( argc < 3) {
      printf("Require two files\n");
      exit(1);
      
   }
compareFiles(argv[1], argv[2]);
}


Overwriting compareResults.cu


In [31]:
!nvcc compareResults.cu -o compareResults

## 1) Sequential Execution

### Source Code

In [32]:
%%writefile sequential.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define XNOR 5

#define LINE_SIZE 16

// Container for our working data, single Task
typedef struct Task {
    char gate;
    char input1;
    char input2;
    char output;
} Task;


/**
 * @brief Read the given filename, populate inputs 1 and 2
 * of our Task pointer array. Assumes each line 
 * to be LINE_SIZE characters long.
 * 
 * @param input_file String name of the file to read
 * @param length Number of lines in the file
 * @param tasks Pointer to our Task pointer array
 */
void readInput(char* input_file, int length, Task* tasks) {
    FILE* f = fopen(input_file, "r");

    char line[LINE_SIZE]; // char array length 16
    int i = 0;

    while (fgets(line, LINE_SIZE, f)) {
        if (i >= length) break;
        // convert to ints with - '0'
        tasks[i].input1 = line[0] - '0';
        tasks[i].input2 = line[2] - '0';
        tasks[i].gate = line[4] - '0';
        i++;
    }

    fclose(f);
}

/**
 * @brief Produces the output file containing a single column of
 * all the logic gate outputs from the tasks array.
 * 
 * @param output_file Name of the file to write.
 * @param length Number of tasks
 * @param tasks Pointer to our Task pointer array
 */
void writeOutput(char* output_file, int length, Task* tasks) {
    FILE* f = fopen(output_file, "w+"); // creates the file if not exists

    for (int i = 0; i < length; i++) {
        // Adding + '0' to convert into char
        fputc(tasks[i].output + '0', f);
        fputc('\n', f);
    }

    fclose(f);
}

/**
 * @brief 
 * Computes the logic gate result for a given list of
 * tasks consisting of two binary inputs, stores in place.
 * @param tasks an array of pointers to Task structs
 * @param input_length the number of tasks to compute
 */
void sequential(Task* tasks, int input_length) {
    for (int i = 0; i < input_length; i++) {
        switch (tasks[i].gate) {
            case AND:
                tasks[i].output = tasks[i].input1 && tasks[i].input2;
                break;
            case OR:
                tasks[i].output = tasks[i].input1 || tasks[i].input2;
                break;
            case NAND:
                tasks[i].output = !(tasks[i].input1 && tasks[i].input2);
                break;
            case NOR:
                tasks[i].output = !(tasks[i].input1 || tasks[i].input2);
                break;
            case XOR:
                tasks[i].output = (tasks[i].input1 || tasks[i].input2) && !(tasks[i].input1 && tasks[i].input2);
                break;
            case XNOR:
                tasks[i].output = (tasks[i].input1 && tasks[i].input2) || (!tasks[i].input1 && !tasks[i].input2);
                break;
        }
    }
}


int main(int argc, char *argv[])
{
    // ============
    // RUN COMMAND:
    // ./sequential <input_file_path> <input_file_length> <output_file_path> 
    // ============

    if (argc != 4) {
        printf("\nIncorrect arguments. Run the program as follows:\n");
        printf("./sequential <input_file_path> <input_file_length> <output_file_path>\n");
    }

    int n_inputs;
    sscanf(argv[2], "%d", &n_inputs);

    // Allocate an array for our Tasks
    Task* tasks = (Task*) malloc(n_inputs * sizeof(Task)); // returns pointer to a task

    // Populate the array in-place
    readInput(argv[1], n_inputs, tasks);

    // simulate logic gates sequentially     
    clock_t startTime = clock();
    sequential(tasks, n_inputs);
    clock_t stopTime = clock();
    // CLOCKS_PER_SEC is the number of clock tics elapsed in a second, 
    // which gives us the precise CPU time consumed by a task
    double totalTimeInMs = (((double)stopTime - (double)startTime) * 1000.0) / CLOCKS_PER_SEC;
    // print total time elapsed for sequential gate simulation
    printf("Sequential Simultation: %f ms\n", totalTimeInMs);
    
    writeOutput(argv[3], n_inputs, tasks);
    
    free(tasks);

    return 0;
}

Overwriting sequential.cu


### Compile and Run

In [33]:
!nvcc sequential.cu -o sequential

In [34]:
!./sequential input_10000.txt 10000 output_10000.txt 

Sequential Simultation: 0.280000 ms


In [35]:
!./sequential input_100000.txt 100000 output_100000.txt 

Sequential Simultation: 2.737000 ms


In [36]:
!./sequential input_1000000.txt 1000000 output_1000000.txt 

Sequential Simultation: 23.612000 ms


### Verify Results
Use `./compareResults <path_to_solution> <path_to_output>`

In [37]:
!./compareResults sol_10000.txt output_10000.txt

Total Errors : 0	

In [38]:
!./compareResults sol_100000.txt output_100000.txt

Total Errors : 0	

In [39]:
!./compareResults sol_1000000.txt output_1000000.txt

Total Errors : 0	

## 2) Parallelized with Explicit Memory Allocation

### Source Code

In [40]:
%%writefile parallel_explicit.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// CUDA runtime
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#ifndef __GPU_TIMER_H__
#define __GPU_TIMER_H__
#endif

#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define XNOR 5

#define LINE_SIZE 16
#define THREADS_PER_BLOCK 1024

// Macro for Cuda Debugging
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

// Container for our working data, single Task
typedef struct Task {
    char gate;
    char input1;
    char input2;
    char output;
} Task;

/**
 * @brief Read the given filename, populate inputs 1 and 2
 * of our Task pointer array. Assumes each line 
 * to be LINE_SIZE characters long.
 * 
 * @param input_file String name of the file to read
 * @param length Number of lines in the file
 * @param tasks Pointer to our Task pointer array
 */
void readInput(char* input_file, int length, Task* tasks) {
    FILE* f = fopen(input_file, "r");

    char line[LINE_SIZE]; // char array length 16
    int i = 0;

    while (fgets(line, LINE_SIZE, f)) {
        if (i >= length) break;
        // convert to ints with - '0'
        tasks[i].input1 = line[0] - '0';
        tasks[i].input2 = line[2] - '0';
        tasks[i].gate = line[4] - '0';
        i++;
    }

    fclose(f);
}

/**
 * @brief Produces the output file containing a single column of
 * all the logic gate outputs from the tasks array.
 * 
 * @param output_file Name of the file to write.
 * @param length Number of tasks
 * @param tasks Pointer to our Task pointer array
 */
void writeOutput(char* output_file, int length, Task* tasks) {
    FILE* f = fopen(output_file, "w+"); // creates the file if not exists

    for (int i = 0; i < length; i++) {
        // Adding + '0' to convert into char
        fputc(tasks[i].output + '0', f);
        fputc('\n', f);
    }

    fclose(f);
}

/**
 * @brief 
 * Computes the logic gate result for a given task
 * consisting of two binary inputs, stores in place.
 * @param tasks an array of pointers to Task structs
 */
__global__ void parallel_explicit(Task* tasks, int n_inputs) {
    int i = (blockIdx.x * blockDim.x) + threadIdx.x;
    if (i < n_inputs) {
        switch (tasks[i].gate) {
                case AND:
                    tasks[i].output = tasks[i].input1 && tasks[i].input2;
                    break;
                case OR:
                    tasks[i].output = tasks[i].input1 || tasks[i].input2;
                    break;
                case NAND:
                    tasks[i].output = !(tasks[i].input1 && tasks[i].input2);
                    break;
                case NOR:
                    tasks[i].output = !(tasks[i].input1 || tasks[i].input2);
                    break;
                case XOR:
                    tasks[i].output = (tasks[i].input1 || tasks[i].input2) && !(tasks[i].input1 && tasks[i].input2);
                    break;
                case XNOR:
                    tasks[i].output = (tasks[i].input1 && tasks[i].input2) || (!tasks[i].input1 && !tasks[i].input2);
                    break;
            }
    }
}

int main(int argc, char *argv[])
{
    // ============
    // RUN COMMAND:
    // ./parallel_explicit <input_file_path> <input_file_length> <output_file_path> 
    // ============

    if (argc != 4) {
        printf("\nIncorrect arguments. Run the program as follows:\n");
        printf("./parallel_explicit <input_file_path> <input_file_length> <output_file_path>\n");
    }

    int n_inputs;
    sscanf(argv[2], "%d", &n_inputs); // n_threads

    size_t TASKS_SIZE = n_inputs * sizeof(Task);
    // Allocate an array for our Tasks
    Task* tasks = (Task*) malloc(TASKS_SIZE); // returns pointer to a task

    // Populate the array in-place
    readInput(argv[1], n_inputs, tasks);

    // Allocate memory and copy to the device
    Task* d_tasks;
    gpuErrchk(cudaMalloc((void **)&d_tasks, TASKS_SIZE));
    gpuErrchk(cudaMemcpy(d_tasks, tasks, TASKS_SIZE, cudaMemcpyHostToDevice));

    // KERNEL LAUNCH

    float memsettime;
    cudaEvent_t start, stop;
    gpuErrchk(cudaEventCreate(&start));
    gpuErrchk(cudaEventCreate(&stop));
    gpuErrchk(cudaEventRecord(start, 0));
    
    parallel_explicit<<<ceil(n_inputs/THREADS_PER_BLOCK) + 1, THREADS_PER_BLOCK>>>(d_tasks, n_inputs); // n threads for n tasks
    cudaDeviceSynchronize();

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&memsettime, start, stop);
    printf("Parallel w/ Explicit Allocation: %f ms\n", memsettime);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    // Copy results back to CPU memory
    cudaMemcpy(tasks, d_tasks, TASKS_SIZE, cudaMemcpyDeviceToHost);

    cudaFree(d_tasks);

    writeOutput(argv[3], n_inputs, tasks);
    free(tasks);

    return 0;
}

Overwriting parallel_explicit.cu


### Compile and Run

In [41]:
!nvcc parallel_explicit.cu -o parallel_explicit

In [42]:
!./parallel_explicit input_10000.txt 10000 output_pll_explicit_10000.txt 

Parallel w/ Explicit Allocation: 0.026624 ms


In [43]:
!./parallel_explicit input_100000.txt 100000 output_pll_explicit_100000.txt 

Parallel w/ Explicit Allocation: 0.031008 ms


In [44]:
!./parallel_explicit input_1000000.txt 1000000 output_pll_explicit_1000000.txt 

Parallel w/ Explicit Allocation: 0.138624 ms


### Verify Results

In [45]:
!./compareResults sol_10000.txt output_pll_explicit_10000.txt

Total Errors : 0	

In [46]:
!./compareResults sol_100000.txt output_pll_explicit_100000.txt

Total Errors : 0	

In [47]:
!./compareResults sol_1000000.txt output_pll_explicit_1000000.txt

Total Errors : 0	

## 3) Parallelized with Managed (Unified) Memory Allocation

### Source Code

In [48]:
%%writefile parallel_unified.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// CUDA runtime
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#ifndef __GPU_TIMER_H__
#define __GPU_TIMER_H__
#endif

#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define XNOR 5

#define LINE_SIZE 16
#define THREADS_PER_BLOCK 1024

// Macro for Cuda Debugging
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

// Container for our working data, single Task
typedef struct Task {
    char gate;
    char input1;
    char input2;
    char output;
} Task;

/**
 * @brief Read the given filename, populate inputs 1 and 2
 * of our Task pointer array. Assumes each line 
 * to be LINE_SIZE characters long.
 * 
 * @param input_file String name of the file to read
 * @param length Number of lines in the file
 * @param tasks Pointer to our Task pointer array
 */
void readInput(char* input_file, int length, Task* tasks) {
    FILE* f = fopen(input_file, "r");

    char line[LINE_SIZE]; // char array length 16
    int i = 0;

    while (fgets(line, LINE_SIZE, f)) {
        if (i >= length) break;
        // convert to ints with - '0'
        tasks[i].input1 = line[0] - '0';
        tasks[i].input2 = line[2] - '0';
        tasks[i].gate = line[4] - '0';
        i++;
    }

    fclose(f);
}

/**
 * @brief Produces the output file containing a single column of
 * all the logic gate outputs from the tasks array.
 * 
 * @param output_file Name of the file to write.
 * @param length Number of tasks
 * @param tasks Pointer to our Task pointer array
 */
void writeOutput(char* output_file, int length, Task* tasks) {
    FILE* f = fopen(output_file, "w+"); // creates the file if not exists

    for (int i = 0; i < length; i++) {
        // Adding + '0' to convert into char
        fputc(tasks[i].output + '0', f);
        fputc('\n', f);
    }

    fclose(f);
}

/**
 * @brief 
 * Computes the logic gate result for a given task
 * consisting of two binary inputs, stores in place.
 * @param tasks an array of pointers to Task structs
 */
__global__ void parallel_unified(Task* tasks, int n_inputs) {
    int i = (blockIdx.x * blockDim.x) + threadIdx.x;
    if (i < n_inputs) {
        switch (tasks[i].gate) {
                case AND:
                    tasks[i].output = tasks[i].input1 && tasks[i].input2;
                    break;
                case OR:
                    tasks[i].output = tasks[i].input1 || tasks[i].input2;
                    break;
                case NAND:
                    tasks[i].output = !(tasks[i].input1 && tasks[i].input2);
                    break;
                case NOR:
                    tasks[i].output = !(tasks[i].input1 || tasks[i].input2);
                    break;
                case XOR:
                    tasks[i].output = (tasks[i].input1 || tasks[i].input2) && !(tasks[i].input1 && tasks[i].input2);
                    break;
                case XNOR:
                    tasks[i].output = (tasks[i].input1 && tasks[i].input2) || (!tasks[i].input1 && !tasks[i].input2);
                    break;
            }
    }
}

int main(int argc, char *argv[])
{
    // ============
    // RUN COMMAND:
    // ./parallel_unified <input_file_path> <input_file_length> <output_file_path> 
    // ============

    if (argc != 4) {
        printf("\nIncorrect arguments. Run the program as follows:\n");
        printf("./parallel_unified <input_file_path> <input_file_length> <output_file_path>\n");
    }

    int n_inputs;
    sscanf(argv[2], "%d", &n_inputs); // n_threads

    size_t TASKS_SIZE = n_inputs * sizeof(Task);
    // Allocate an array for our Tasks, accesible from both CPU and GPU
    Task* tasks;
    gpuErrchk(cudaMallocManaged((void **)&tasks, TASKS_SIZE));
    // Populate the array in-place
    readInput(argv[1], n_inputs, tasks);

    float memsettime;
    cudaEvent_t start, stop;
    gpuErrchk(cudaEventCreate(&start));
    gpuErrchk(cudaEventCreate(&stop));
    gpuErrchk(cudaEventRecord(start, 0));
    
    parallel_unified<<<ceil(n_inputs/THREADS_PER_BLOCK) + 1, THREADS_PER_BLOCK>>>(tasks, n_inputs); // n threads for n tasks
    cudaDeviceSynchronize();

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&memsettime, start, stop);
    printf("Parallel w/ Unified Memory Allocation: %f ms\n", memsettime);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    writeOutput(argv[3], n_inputs, tasks);

    cudaFree(tasks);
    return 0;
}

Overwriting parallel_unified.cu


### Compile and Run

In [49]:
!nvcc parallel_unified.cu -o parallel_unified

In [50]:
!./parallel_unified input_10000.txt 10000 output_pll_unified_10000.txt 

Parallel w/ Unified Memory Allocation: 0.339872 ms


In [51]:
!./parallel_unified input_100000.txt 100000 output_pll_unified_100000.txt

Parallel w/ Unified Memory Allocation: 0.536800 ms


In [52]:
!./parallel_unified input_1000000.txt 1000000 output_pll_unified_1000000.txt

Parallel w/ Unified Memory Allocation: 1.781824 ms


### Verify Results

In [53]:
!./compareResults sol_10000.txt output_pll_unified_10000.txt

Total Errors : 0	

In [54]:
!./compareResults sol_100000.txt output_pll_unified_100000.txt

Total Errors : 0	

In [55]:
!./compareResults sol_1000000.txt output_pll_unified_1000000.txt

Total Errors : 0	